In [1]:
conda install geopandas

Solving environment: done

## Package Plan ##

  environment location: /Users/DBurke/opt/miniconda3

  added / updated specs:
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.7.22  |                0         124 KB
    conda-4.8.4                |           py37_0         2.9 MB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  ca-certificates                               2020.6.24-0 --> 2020.7.22-0
  conda                                        4.8.3-py37_0 --> 4.8.4-py37_0



conda-4.8.4          | 2.9 MB    | ##################################### | 100% 
ca-certificates-2020 | 124 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: y

In [7]:
import fiona
import dash
import geopandas as gpd
import simplejson as json
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import ast
import numpy as np
import sqlite3
import fb
import datetime
import sys
from geopandas.tools import sjoin
import warnings; warnings.simplefilter('ignore')
import datetime
from datetime import timedelta

In [8]:
db = '/Users/DBurke/Documents/Layerlab/COvid19/dbs/boulder.db'
print(db)

/Users/DBurke/Documents/Layerlab/COvid19/dbs/boulder.db


In [9]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [34]:

#dates = ['20200830', '20200831', '20200901', '20200902', '20200903']
#dates = ['20200821','20200820','20200819','20200818','20200817','20200816','20200815','20200814','20200813','20200812','20200811','20200810','20200809','20200808','20200807','20200806','20200805','20200804','20200803','20200802','20200801','20200731','20200730']
#dates.reverse()
dates = []
start_dt = datetime.date(2020, 8, 10)
end_dt = datetime.date(2020, 8, 10)

for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y%m%d"))
df_temp = pd.DataFrame()
dataframes_list = []
for today in dates:
    print(today)
    df = pd.DataFrame()
    sit_rep_name = 'Boulder'
    base_path = "/Users/DBurke/Documents/Layerlab/COvid19" #CHANGE PATH
    sit_rep_path = base_path+'/sitreps/'+sit_rep_name+'/'+today +'/'
    location_data_path = base_path + "/dash/location_data/"
    src = base_path+'/src'
    latest_county_scores = sit_rep_path + sit_rep_name + '_county_scores.' + today + '.txt'
    latest_city_scores = sit_rep_path + sit_rep_name + '_city_scores.' + today + '.txt'
    hot_spot_city_shape = sit_rep_path + sit_rep_name + '_hot_spot_city_shape'
    hot_spot_county_shape = sit_rep_path + sit_rep_name + '_hot_spot_county_shape/'
    
    geodf = gpd.read_file(hot_spot_county_shape+ sit_rep_name +"_hot_spot_county_shape.shp")
    geodf.to_file(sit_rep_path + sit_rep_name + '_hotspot_geojson.geojson', driver = "GeoJSON")
    j_file = geodf.to_json()
    j_file = ast.literal_eval(j_file)

    coordinates = []
    z_score = []
    new_jfile = {}

    new_jfile['type'] = j_file['type']
    new_jfile['features'] = []
    
    for i in range(len(j_file['features'])):
        new_jfile['features'].append(j_file['features'][i])
        coordinates.append(j_file['features'][i]['geometry']['coordinates'])
        z_score.append(j_file['features'][i]['properties']['z'])

    df['coordinates_'+today] = coordinates
    df['z_score_'+today] = z_score
    

    new_jfile['features'][0].keys()
    for k in range(len(new_jfile['features'])):
        new_jfile['features'][k]['id'] = k
    ids=[]
    new_jfile['features'][0].keys()
    for k in range(len(new_jfile['features'])):
        ids.append(new_jfile['features'][k]['id'])
    df['ids'] = ids
    dataframes_list.append(df)

df_temp = pd.concat(dataframes_list)
df = df_temp



20200810


In [35]:
#mapbox_access_token = open("COvid19/dash/.mapbox_token").read()
mapbox_access_token = 'pk.eyJ1IjoicnlhbmxheWVyIiwiYSI6ImNrOW9mZm1kcDAwMXczZG8xZGN2eWJsaWwifQ.-Ac4FRJlrzHzsyo4fmXUGA'
print(mapbox_access_token)

pk.eyJ1IjoicnlhbmxheWVyIiwiYSI6ImNrOW9mZm1kcDAwMXczZG8xZGN2eWJsaWwifQ.-Ac4FRJlrzHzsyo4fmXUGA


In [36]:
'''
fig = go.Figure(go.Choroplethmapbox(z=df['z_score'],
                            locations = df['ids'], 
                            colorscale = 'hot_r',
                            colorbar = dict(thickness=20, ticklen=3),
                            geojson = new_jfile,
                            text = df['coordinates'],
                            hovertemplate = '<br> <b>Hot Spot </b>: %{z}<br>',
                            marker_line_width=0.1, marker_opacity=0.7))

fig.update_layout(title_text ='Norway mapbox choropleth', title_x =0.5, width=750, height=700,
                 mapbox = dict(center= dict(lat=64.5, lon=18.75),            
                               accesstoken= mapbox_access_token,
                               zoom=3
                               ))

fig.show()
'''



"\nfig = go.Figure(go.Choroplethmapbox(z=df['z_score'],\n                            locations = df['ids'], \n                            colorscale = 'hot_r',\n                            colorbar = dict(thickness=20, ticklen=3),\n                            geojson = new_jfile,\n                            text = df['coordinates'],\n                            hovertemplate = '<br> <b>Hot Spot </b>: %{z}<br>',\n                            marker_line_width=0.1, marker_opacity=0.7))\n\nfig.update_layout(title_text ='Norway mapbox choropleth', title_x =0.5, width=750, height=700,\n                 mapbox = dict(center= dict(lat=64.5, lon=18.75),            \n                               accesstoken= mapbox_access_token,\n                               zoom=3\n                               ))\n\nfig.show()\n"

In [37]:
frames = [{   
    'name':'frame_{}'.format(today),
    'data':[{
        'type':'choroplethmapbox',
        'z':df['z_score_'+today],
        'locations' : df['ids'], 
        'colorscale' : 'hot_r',
        'colorbar' : dict(thickness=20, ticklen=3),
        'geojson' : new_jfile,
        'text' : df['coordinates_'+today],
        'hovertemplate' : '<br> <b>Hot Spot </b>: %{z}<br>',
        'marker_line_width':0.1, 
        'marker_opacity': 0.7
    }],           
} for today in dates]  

In [38]:

sliders = [{
    'transition':{'duration': 5},
    'x':0.08, 
    'len':0.88,
    'currentvalue':{'font':{'size':15}, 'visible':True, 'xanchor':'center'},  
    'steps':[
        {
            'label':today,
            'method':'animate',
            'args':[
                ['frame_{}'.format(today)],
                {'mode':'immediate', 'frame':{'duration':1000, 'redraw': True}, 
                 'transition':{'duration':150}}
              ],
        } for today in dates]
}]


In [39]:
play_button = [{
    'type':'buttons',
    'buttons':[{ 
        'label':'Play', # Play
        'method':'animate',
        'args':[None]
    }]
}],

In [67]:
print('starting')
data = frames[0]['data']
#pos= ('40.008683404152', '-105.28472900391')
# Adding all sliders and play button to the layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=[{
    'type':'buttons',
    'showactive':True,
    'x':0.045, 'y':-0.08,
    'buttons':[{ 
        'label':'Play', # Play
        'method':'animate',
        'args':[
            None,
            {
                'frame':{'duration':1000, 'redraw':True},
                'transition':{'duration':150},
                'fromcurrent':True,
                'mode':'immediate',
            }
        ]
    }]
}],
    mapbox={
        'accesstoken':mapbox_access_token,
        'center':{"lat": 40.0086, "lon": -105.28},
        'zoom':6,
        'style':'light',
    }
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)


######update figure
fig.update_layout(title_x =0.5, width=1250, height=1200,
                 mapbox = dict(center= dict(lat=40.0086, lon=-105.275),            
                               accesstoken= mapbox_access_token,
                               zoom=11
                               ))

fig.update_layout(mapbox = dict(accesstoken= mapbox_access_token,
                               layers = [ dict(
                                   type = "symbol",
                                   color = "#2600ff",
                                   coordinates = [40.0086, -105.28]
                               )]))
                                         

# Displaying the figure

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))
fig.show()

starting
